In [1]:
import Pkg; Pkg.add("RobotZoo"); Pkg.add("StaticArrays")
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
using LinearAlgebra
using Plots; plotly()
using Random
using JLD2
using Test
using MeshCat
const mc = MeshCat
# using TrajOptPlots
# using StaticArrays
using Printf
using RobotZoo
using StaticArrays

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/Dropbox/My Mac (MacBook Pro (2))/Desktop/CMU/Optimal Control/Optimal-Controls-Gymnastics-Bot/MonkeyBarBot/Project.toml`
  No Changes to `~/Dropbox/My Mac (MacBook Pro (2))/Desktop/CMU/Optimal Control/Optimal-Controls-Gymnastics-Bot/MonkeyBarBot/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Dropbox/My Mac (MacBook Pro (2))/Desktop/CMU/Optimal Control/Optimal-Controls-Gymnastics-Bot/MonkeyBarBot/Project.toml`
  No Changes to `~/Dropbox/My Mac (MacBook Pro (2))/Desktop/CMU/Optimal Control/Optimal-Controls-Gymnastics-Bot/MonkeyBarBot/Manifest.toml`
  Activating environment at `~/Dropbox/My Mac (MacBook Pro (2))/Desktop/CMU/Optimal Control/Optimal-Controls-Gymnastics-Bot/MonkeyBarBot/Project.toml`
┌ Warning: backend `PlotlyKaleido` is not installed.
└ @ Plots ~/.julia/packages/Plots/B5j7d/src/backends.jl:43


In [2]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils", "monkeyBotDynamics.jl"))
include(joinpath(@__DIR__, "utils","walker.jl"))

update_walker_pose! (generic function with 1 method)

In [3]:
model = (g = 9.81, mb= 5.0, mf = 1.0, ℓ_min = 0.5, ℓ_max = 1.5,L1 = 1.0,L2 = 1.0)
N = 20
X = Array{Any}(undef,N)
c = Array{Any}(undef,N)
c[1] = 1
Xn = Vector([1.0,1.0,1.0,0.0,1.0,0.0])
L = 1.0
Θ = [0.0,0.0]

# X2 = 
X[1] = Xn
X[2] = -Xn
@show X[1]
dt = 0.1
t = collect(0:dt:N*dt)

X[1] = [1.0, 1.0, 1.0, 0.0, 1.0, 0.0]


21-element Vector{Float64}:
 0.0
 0.1
 0.2
 0.3
 0.4
 0.5
 0.6
 0.7
 0.8
 0.9
 1.0
 1.1
 1.2
 1.3
 1.4
 1.5
 1.6
 1.7
 1.8
 1.9
 2.0

In [4]:
L1 = 1.0
L2 = 1.0

for i=2:10
    theta = [(3.0*t[i]),pi/2]
    px = 0.0
    py = 0.0
    Xn = Vector([theta[1],theta[2],px,py])
    X[i] = Xn
    c[i] = 1
end

for i=11:N
    theta = [pi/2,(3.0*t[i])]
    px = 0.0
    py = 0.0
    Xn = Vector([theta[1],theta[2],px,py])
    X[i] = Xn
    c[i] = -1
end

@show c

c = Any[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


20-element Vector{Any}:
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1

In [5]:
vis = Visualizer()
build_walker!(vis, model::NamedTuple)
anim = mc.Animation(floor(Int,1/dt))
params = (l1 = 1.0,l2 = 1.0)
for k = 1:N
    contact_mode = c[k]
#     switch contact mode , leftarm =1,rightarm = -1
    mc.atframe(anim, k) do
        update_monkey_pose!(vis,contact_mode,model::NamedTuple,params::NamedTuple, X[k])
    end
end
mc.setanimation!(vis, anim)
display(render(vis))

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8701


LoadError: UndefVarError: xb not defined